In [43]:
import pandas as pd
from pathlib import Path
from pandas.tseries.offsets import MonthEnd
import numpy as np

In [44]:
monthly_path = Path.cwd() / "data" / "monthly.csv"
dtypes = {
    "category": str,
    "expense_type": str,
    "broad_type": str,
    "actuals": float,
    "to_month": str,
    "to_year": str,
    "monthly_expected": float
}
monthly = pd.read_csv(monthly_path, dtype=dtypes)

In [63]:
monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   category          27 non-null     object 
 1   expense_type      27 non-null     object 
 2   broad_type        27 non-null     object 
 3   actuals           24 non-null     float64
 4   to_month          22 non-null     object 
 5   to_year           22 non-null     object 
 6   monthly_expected  18 non-null     float64
dtypes: float64(2), object(5)
memory usage: 1.6+ KB


In [64]:
import math


def render_date(df):
    df = df.rename(columns={"to_month": "month", "to_year": "year"})
    df["to_date"] = pd.to_datetime(df[["year", "month"]].assign(DAY=1), errors="ignore")
    df["to_date"] = df["to_date"] + MonthEnd(1)
    df = df.drop(columns=["month", "year"])
    return df


def count_months(row):
    row["last_month_of_year"] = row["to_date"].replace(month=12) + MonthEnd(0)
    row["remaining_months"] = (
        row["last_month_of_year"] - row["to_date"]
    ) / np.timedelta64(1, "M")
    row["remaining_months"] = (
        int(row["remaining_months"])
        if not math.isnan(row["remaining_months"])
        else np.nan
    )
    row = row.drop(columns=["last_month_of_year"])
    return row


def compute_totals(row):
    row["expected_amount"] = row["monthly_expected"] * row["remaining_months"]
    if math.isnan(row["actuals"]):
        row["total"] = row["expected_amount"]
    elif math.isnan(row["expected_amount"]):
        row["total"] = row["actuals"]
    else:
        row["total"] = row["actuals"] + row["expected_amount"]
    return row


df_dates = render_date(monthly)
df_count = df_dates.apply(count_months, axis=1)
test = df_count.apply(compute_totals, axis=1)